### 1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
###2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
###3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
###4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
###5. (опционально) Еще раз провести оценку экономической эффективности

### Case 1

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Есть как категориальные, так и вещественные признаки. Поле CustomerId нужно будет удалить. 

Посмотрим на распределение классов:

In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

- Категориальные признаки закодируем с помощью OneHotEncoding
- Вещественные оставим пока как есть

In [5]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Зададим списки признаков

In [7]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

Посмотрим как это работает на примере отдельного категориального признака - Gender

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

gender = Pipeline([
                ('selector', FeatureSelector(column='Gender')),
                ('ohe', OHEEncoder(key='Gender'))
            ])
gender.fit(X_train)
gender.transform(X_test).head(3)

,Gender_Female,Gender_Male
9394,1,0
898,1,0
2398,1,0


Теперь нам нужно под каждый признак создать трансформер и объединить их в список (сделаем это в цикле, чтобы не мучиться)

In [9]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [10]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Теперь у нас есть пайплайн, который готовит признаки для моделирования.

###Добавим модели

In [11]:
model_types = {'Случайный лес': RandomForestClassifier(random_state = 42),
               'Градиентный бустинг': GradientBoostingClassifier(random_state = 42),
               'Логистичиская регрессия': LogisticRegression(random_state = 42),
              }

models = []
for key in model_types:
    
    classifier = Pipeline([
        ('features',feats),
        ('classifier', model_types[key]),
    ])
    
    model = {'title': key, 'classifier': classifier}
    models.append(model)  

In [12]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

metrics_df = pd.DataFrame(columns=['model', 'Threshold', 'F-Score', 'Precision', 'Recall'])

all_cnf_matrix = []

In [13]:
for model in models:
  classifier = model['classifier']
  
  pipeline = Pipeline([
    ('features',feats),
    ('classifier', classifier),
  ])
  
  #обучим наш пайплайн
  classifier.fit(X_train, y_train)

  #наши прогнозы для тестовой выборки
  preds = classifier.predict_proba(X_test)[:, 1]
  print(preds[:10])

  #рассчитаем метрики и запишем их в датасет
  precision, recall, thresholds = precision_recall_curve(y_test, preds)

  fscore = (2 * precision * recall) / (precision + recall)
  ix = np.argmax(fscore)
  print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
  
  metrics_df = metrics_df.append({
    'model': model['title'],
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix]
    #'Roc auc': roc_auc
  }, ignore_index=True)

  cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
  all_cnf_matrix.append(cnf_matrix) 

[0.37 0.26 0.17 0.02 0.02 0.67 0.04 0.12 0.15 0.75]
Best Threshold=0.380000, F-Score=0.641, Precision=0.654, Recall=0.629


<ipython-input-13-3f5cef0d62b9>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


[0.28502357 0.22926788 0.1982218  0.08930468 0.04798288 0.80662596
 0.03834531 0.05054472 0.31038377 0.73455865]
Best Threshold=0.408508, F-Score=0.646, Precision=0.704, Recall=0.597
[0.22805865 0.3350657  0.15347886 0.12466446 0.15507743 0.6431308
 0.06214346 0.077948   0.36717132 0.76751542]
Best Threshold=0.289522, F-Score=0.510, Precision=0.462, Recall=0.568


<ipython-input-13-3f5cef0d62b9>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({
<ipython-input-13-3f5cef0d62b9>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [14]:
all_cnf_matrix

[array([[1833,  158],
        [ 195,  314]]),
 array([[1863,  128],
        [ 206,  303]]),
 array([[1655,  336],
        [ 221,  288]])]

In [15]:
metrics_df

,model,Threshold,F-Score,Precision,Recall
0,Случайный лес,0.380000,0.641283,0.654397,0.628684
1,Градиентный бустинг,0.408508,0.646121,0.703704,0.597250
2,Логистичиская регрессия,0.289522,0.509700,0.462400,0.567780


###По метрикам отбираем лучшую модель - Градиентный бустинг.
###Определим экономический эффект модели

In [16]:
def get_rates(cnf_matrix):

    TN = cnf_matrix[0][0]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]
    FP = cnf_matrix[0][1]
    return TN, FN, TP, FP

In [17]:
TN, FN, TP, FP = get_rates(all_cnf_matrix[1])
TN, FN, TP, FP
print(f"При оптимальном пороге вероятности  {metrics_df['F-Score'][1]}  ожидаемый экономический эффект составит - {TP*2 - (TP + FP)} долларов")


При оптимальном пороге вероятности  0.6461211477151967  ожидаемый экономический эффект составит - 175 долларов


###Попробуем подобрать параметры к нашей модели, чтобы улучшить метрики и экономический эффект

In [23]:
from sklearn.model_selection import GridSearchCV

parameters = {'learning_rate': [0.15, 0.2, 0.25], 'n_estimators': [80, 100, 110],
              'min_samples_leaf':[1, 2, 3], 'max_depth':[3, 4, 5]}

gb = GradientBoostingClassifier(random_state = 42)
gs = GridSearchCV(gb, parameters)

pipeline = Pipeline([
    ('features', feats),
    ('classifier', gs),
])

pipeline.fit(X_train, y_train)

gs.best_params_, gs.best_score_

({'learning_rate': 0.2,
  'max_depth': 3,
  'min_samples_leaf': 2,
  'n_estimators': 100},
 0.8652)

In [19]:
#Обучим наш пайплайн с параметрами

pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(learning_rate=0.2, max_depth=3, 
                                          min_samples_leaf=2, n_estimators=100, random_state=42)),
])
pipeline.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                                  StandardScaler())])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary')),
                                                                 ('standard',
                                                                  StandardScaler())]))])),
                ('classifier',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            min_samples_leaf=2,
                                            random_state=42))])

In [24]:
preds = pipeline.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])

TN, FN, TP, FP = get_rates(cnf_matrix)
TN, FN, TP, FP
print(f"При оптимальном пороге вероятности  {fscore[ix]}  ожидаемый экономический эффект составит - {TP*2 - (TP + FP)} долларов")

Best Threshold=0.343215, F-Score=0.650, Precision=0.642, Recall=0.658
При оптимальном пороге вероятности  0.649854510184287  ожидаемый экономический эффект составит - 147 долларов
